# Clobber Together Source Data
[Summarize]

In [ ]:
import pandas as pd
import os
import logging
import requests, json
import numpy as np
import gc
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
DATA_DIR = "data/"
CSV_DIR = DATA_DIR + "tripdata_csv/"
NY_DIR = CSV_DIR + "NY/"
NJ_DIR = CSV_DIR + "NJ/"

DB_FILE = "data/tripdata.db"

logging.basicConfig(level=logging.WARNING)

JC_DATA = os.listdir(NJ_DIR)  # NOTE: this includes Hoboken and Jersey City
NYC_DATA = os.listdir(NY_DIR)

logging.info(
    f"{len(JC_DATA)} Jersey City files and {len(NYC_DATA)} New York City files"
)

SCHEMA_CHANGE_DATE = "2021-02"

# CSV paths for NYC, JC (pre and post schema change)
nyc_old = sorted([NY_DIR + f for f in os.listdir(NY_DIR) if f < SCHEMA_CHANGE_DATE])
nyc_new = sorted([NY_DIR + f for f in os.listdir(NY_DIR) if f >= SCHEMA_CHANGE_DATE])

jc_old = sorted([NJ_DIR + f for f in os.listdir(NJ_DIR) if f < SCHEMA_CHANGE_DATE])
jc_new = sorted([NJ_DIR + f for f in os.listdir(NJ_DIR) if f >= SCHEMA_CHANGE_DATE])

# Create Annual Data Tables (rides and stations)

In [ ]:
def clobber_year(year=2019, state="NY") -> pd.DataFrame:
    """
    Creates a dataframe from source CSVs that is all monthly trip data for that `year`

    :param year: the year for which to concatenate data files
    :param state: 'NY' or 'NJ'. default 'NY'
    :return: the merged dataframe
    """

    range_start = str(year) + "-01"
    range_end = str(year) + "-13"  # Not sure why I have to select 13 here...
    files = None
    if state == "NY":
        files = sorted(
            [NY_DIR + f for f in os.listdir(NY_DIR) if range_start <= f <= range_end]
        )
    elif state == "NJ":
        files = sorted(
            [NJ_DIR + f for f in os.listdir(NJ_DIR) if range_start <= f <= range_end]
        )
    else:
        raise IndexError(f"No data for state: {state}")

    logging.debug(f"Will merge these files: {files}, number of files: {len(files)}")

    # Concatenate all monthly data in range
    clobbered = pd.DataFrame()
    coltypes = {
        "tripduration": "int32",
        "starttime": "datetime64",
        "stoptime": "datetime64",
        "startstationid": "category",
        "startstationname": "category",
        "startstationlatitude": "category",
        "startstationlongitude": "category",
        "endstationid": "category",
        "endstationname": "category",
        "endstationlatitude": "category",
        "endstationlongitude": "category",
        "bikeid": "category",
        "usertype": "category",
        "birthyear": "category",
        "gender": "category",
    }
    for file in files:
        print("loading..." + file)
        df = pd.read_csv(file)
        print("formatting columns..." + file)
        df.columns = [col.lower().replace(" ", "") for col in df.columns]
        # for some reason runs faster with dropped NA and converting temp df types
        df.dropna(axis=0, how="any", inplace=True)
        df = df.astype(coltypes)
        print("concating..." + file)
        clobbered = pd.concat([clobbered, df], axis=0, ignore_index=True)
        del df
        gc.collect()
        print("unloaded..." + file)

    # update dtypes - category conversion lost on concat
    clobbered = clobbered.astype(coltypes)
    print(year, "...dtypes converted")

    return clobbered

In [ ]:
def gen_year_data_files(years=[2019], state="NY"):
    """
    Calls clobber_year, separates ride and station data, writes output to both csv and parquet to `data/`

    :param years: list of years to generate data files for
    :param state: 'NY' or 'NJ'. default 'NY'
    :return: nothing
    """

    for year in years:
        # clobber dataframe
        print("clobbering...", year)
        rides = clobber_year(year, state)
        gc.collect()
        print(year, "...clobbered")

        # extract station data and save to file
        print("extracting stations...", year)
        stations = pd.DataFrame()
        col_select = [
            "startstationid",
            "startstationname",
            "startstationlatitude",
            "startstationlongitude",
        ]
        stations = rides[col_select]
        col_rename = {
            "startstationid": "stationid",
            "startstationname": "stationname",
            "startstationlatitude": "latitude",
            "startstationlongitude": "longitude",
        }
        stations.rename(columns=col_rename, inplace=True)
        stations.drop_duplicates(subset=["stationid"], inplace=True)
        exportpath = "data/stations_" + str(year) + ".parquet"
        stations.to_parquet(exportpath)
        del stations
        gc.collect()
        print(year, "...stations extracted & saved")

        # remove uneeded cols from rides
        drop_cols = [
            "startstationname",
            "startstationlatitude",
            "startstationlongitude",
            "endstationname",
            "endstationlatitude",
            "endstationlongitude",
        ]
        rides.drop(drop_cols, axis=1, inplace=True)

        # save ride to file
        exportpath = "data/rides_" + str(year) + ".parquet"
        rides.to_parquet(exportpath)
        print(year, "...saved to parquet")

        # exportpath = "data/rides_" + str(year) + '.csv'
        # rides.to_csv(exportpath)
        # print(year,'...saved to csv')

        # unload rides dataframe
        del rides
        gc.collect()
        print(year, "...unloaded")

In [ ]:
state = "NY"
years = [2019, 2018, 2017, 2016, 2015]

gen_year_data_files(years, state)

# error when converting 2014 datatypes (did not save files)
# ArrowInvalid: ('Could not convert 1899 with type str: tried to convert to double', 'Conversion failed for column birthyear with type category')
# have not investigated further

# Create Master Rides Table

In [ ]:
def clobber_master_rides(years=[2019]):

    files = ["data/rides_" + str(y) + ".parquet" for y in years]
    coltypes = {
        "tripduration": "int32",
        "starttime": "datetime64",
        "stoptime": "datetime64",
        "startstationid": "category",
        "endstationid": "category",
        "bikeid": "category",
        "usertype": "category",
        "birthyear": "category",
        "gender": "category",
    }

    # concatenate all years data in range
    clobbered = pd.DataFrame()
    for file in files:
        print("loading..." + file)
        df = pd.read_parquet(file)
        print("formatting columns..." + file)
        df = df.astype(coltypes)
        print("concating..." + file)
        clobbered = pd.concat([clobbered, df], axis=0, ignore_index=True)
        del df
        gc.collect()
        print("unloaded..." + file)

    # update dtypes - category conversion lost on concat
    print("converting master dtypes")
    clobbered = clobbered.astype(coltypes)
    print("master dtypes converted")

    # export to file
    print("exporting master rides")
    exportpath = "data/rides_master.parquet"
    clobbered.to_parquet(exportpath)
    print("master rides saved to parquet")

    # clean up
    del clobbered
    gc.collect()

In [ ]:
years = [2019, 2018, 2017, 2016, 2015]
clobber_master_rides(years)

loading...data/rides_2019.parquet
formatting columns...data/rides_2019.parquet
concating...data/rides_2019.parquet
unloaded...data/rides_2019.parquet
loading...data/rides_2018.parquet
formatting columns...data/rides_2018.parquet
concating...data/rides_2018.parquet
unloaded...data/rides_2018.parquet
loading...data/rides_2017.parquet
formatting columns...data/rides_2017.parquet
concating...data/rides_2017.parquet
unloaded...data/rides_2017.parquet
loading...data/rides_2016.parquet
formatting columns...data/rides_2016.parquet
concating...data/rides_2016.parquet
unloaded...data/rides_2016.parquet
loading...data/rides_2015.parquet
formatting columns...data/rides_2015.parquet
concating...data/rides_2015.parquet
unloaded...data/rides_2015.parquet
converting master dtypes
master dtypes converted
exporting master rides
master rides saved to parquet


# Create Master Stations Data Table

In [ ]:
def clobber_master_stations(years=[2019]):

    files = ["data/stations_" + str(y) + ".parquet" for y in years]

    # concatenate all years data in range
    clobbered = pd.DataFrame()
    for file in files:
        print("loading..." + file)
        df = pd.read_parquet(file)
        print("formatting..." + file)
        df.drop_duplicates(subset=["stationid"], inplace=True)
        df = df.astype("category")
        print("concating..." + file)
        clobbered = pd.concat([clobbered, df], axis=0, ignore_index=True)
        del df
        gc.collect()
        print("unloaded..." + file)

    # update dtypes and drop duplicates
    print("formatting master file")
    clobbered.drop_duplicates(subset=["stationid"], inplace=True)
    clobbered = clobbered.astype("category")
    print("formatting complete")

    # load external station details
    print("loading external station details")
    url = requests.get("https://gbfs.citibikenyc.com/gbfs/en/station_information.json")
    text = url.text
    data = json.loads(text)
    station_details = pd.DataFrame.from_dict(data["data"]["stations"])

    # extract capacity and merge back to dataframe
    print("extracting capacity")
    station_details = station_details[["name", "capacity"]]
    station_details.rename(columns={"name": "stationname"}, inplace=True)
    station_details = station_details.astype("category")
    clobbered = clobbered.merge(station_details, how="left", on="stationname")

    # pull geolocation data for each station
    print("reverse geocoding boro and neighbourhood, wait 15-20 mins...")
    geolocator = Nominatim(user_agent="bikegeocode")
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=0)
    locations_lst = []
    for index, row in clobbered.iterrows():
        locations_lst.append(
            reverse("{}, {}".format(row["latitude"], row["longitude"])).raw["address"]
        )
    print("geocode complete, merging...")
    locations = pd.DataFrame(locations_lst, index=clobbered.stationid).reset_index()
    locations = locations[["stationid", "neighbourhood", "suburb", "postcode"]]
    locations.rename(columns={"suburb": "boro", "postcode": "zipcode"}, inplace=True)
    locations = locations.astype("category")
    clobbered = clobbered.merge(locations, how="left", on="stationid")

    # export to file
    print("exporting master stations")
    exportpath = "data/stations_master.parquet"
    clobbered.to_parquet(exportpath)
    print("master stations saved to parquet")

In [ ]:
years = [2019, 2018]  # , 2017, 2016, 2015]
clobber_master_stations(years)

# there are some missing capacities

loading...data/stations_2019.parquet
formatting...data/stations_2019.parquet
concating...data/stations_2019.parquet
unloaded...data/stations_2019.parquet
loading...data/stations_2018.parquet
formatting...data/stations_2018.parquet
concating...data/stations_2018.parquet
unloaded...data/stations_2018.parquet
formatting master file
formatting complete
loading external station details
extracting capacity
reverse geocoding boro and neighbourhood, wait 15-20 mins...
geocode complete, merging...
exporting master stations
master stations saved to parquet


In [ ]:
stations = pd.read_parquet("data/stations_master.parquet")

In [ ]:
stations.head()

,stationid,stationname,latitude,longitude,capacity,neighbourhood,suburb,postcode
0,3160.0,Central Park West & W 76 St,40.778968,-73.973747,39.0,NaN,Manhattan,10023-5104
1,519.0,Pershing Square North,40.751873,-73.977706,NaN,Murray Hill,Manhattan,10037
2,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,39.0,Manhattan Community Board 7,Manhattan,10024
3,504.0,1 Ave & E 16 St,40.732219,-73.981656,54.0,Manhattan Community Board 6,Manhattan,10009
4,229.0,Great Jones St,40.727434,-73.993790,23.0,NoHo,Manhattan,10012


# Create Rebalances Table

# Archive

In [ ]:
bad code  #intentially break if run all cells is performed

# clobber all old nyc CSVs NOTE THIS CRASHES COMPUTER


# nyc_old_dfs = []
# for file in nyc_old:
#     print(f'file {NY_DIR + file}')
#     df = pd.read_csv(NY_DIR + file)
#     nyc_old_dfs.append(df)
#
# nyc_old_df = pd.concat(nyc_old_dfs, axis=0, ignore_index=True)

In [ ]:
# import dask.dataframe as dd
# ddf = dd.read_csv(nyc_old,
#                   dtype={'birth year': 'object',
#                          'end station id': 'float64'})
#
# # columns are Sentence Cased for some CSVs and lower cased for others
# ddf = ddf.rename(columns=str.lower)

In [ ]:
# ddf.describe().compute()

## Monthly Aggregation


In [ ]:
# TODO only works for old schema at the moment
def summarise_months(outfilename: str, months: list):
    """
    Writes monthly summary given list of monthly trip data

    :param outfilename: where to write the summary csv
    :param months: list of CSVs for the monthly trip data
    :return: None
    """
    summaries = []

    for file in months:
        df = pd.read_csv(file)
        df.columns = [col.lower().replace(" ", "") for col in df.columns]
        # logging.debug(f'{file}: {list(df.columns)}')

        year_month = file.split("/")[-1].removesuffix(".csv")  # YYYYMM

        summary = pd.Series(dtype=object)
        summary["datetime"] = year_month
        summary["counttrips"] = df.shape[0]
        summary["meanduration"] = df.tripduration.mean()
        summary["modestartstationid"] = df.startstationid.mode()
        summary["modestartstationname"] = df.startstationname.mode()
        summary["modestartstationlatitude"] = df.startstationlatitude.mode()
        summary["modestartstationlongitude"] = df.startstationlongitude.mode()
        summary["modeendstationid"] = df.endstationid.mode()
        summary["modeendstationname"] = df.endstationname.mode()
        summary["modeendstationlatitude"] = df.endstationlatitude.mode()
        summary["modeendstationlongitude"] = df.endstationlongitude.mode()

        if "usertype" in df.columns:
            summary["usertypevalues"] = df.usertype.value_counts()
        elif "member_casual" in df.columns:
            summary["usertypevalues"] = df.member_casual.value_counts()

        if "gender" in df.columns:
            summary["gendervalues"] = df.gender.value_counts()

        summaries.append(summary)

    summary_df = pd.DataFrame()
    summary_df = summary_df.append(
        summaries
    )  # TODO use concat instead to suppress warning
    summary_df.set_index("datetime")
    summary_df.to_csv(outfilename)


# write summary data month by month for NYC and NJ
summarise_months(DATA_DIR + "summary_nyc_old_schema.csv", nyc_old)
summarise_months(DATA_DIR + "summary_jc_old_schema.csv", jc_old)

# read summary
nyc_old_schema_summary = pd.read_csv("data/summary_nyc_old_schema.csv", index_col=0)
nyc_old_schema_summary

# read JC summary
jc_old_schema_summary = pd.read_csv("data/summary_nyc_old_schema.csv", index_col=0)
jc_old_schema_summary

## Original-ish Clober with logging

In [ ]:
def clobber_year(year=2019, state="NY") -> pd.DataFrame:
    """
    Creates a dataframe from source CSVs that is all monthly trip data for that `year`

    :param year: the year for which to concatenate data files
    :param state: 'NY' or 'NJ'. default 'NY'
    :return: the merged dataframe
    """

    range_start = str(year) + "-01"
    range_end = str(year) + "-13"  # Not sure why I have to select 13 here...
    files = None
    if state == "NY":
        files = sorted(
            [NY_DIR + f for f in os.listdir(NY_DIR) if range_start <= f <= range_end]
        )
    elif state == "NJ":
        files = sorted(
            [NJ_DIR + f for f in os.listdir(NJ_DIR) if range_start <= f <= range_end]
        )
    else:
        raise IndexError(f"No data for state: {state}")

    logging.debug(f"Will merge these files: {files}, number of files: {len(files)}")

    # Concatenate all monthly data in range
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        df.columns = [col.lower().replace(" ", "") for col in df.columns]
        logging.debug(f"Appending df file: {file}...")
        dfs.append(df)
        del df
        gc.collect()
    logging.debug(f"Merging dataframes...")
    clobbered = pd.concat(dfs, axis=0, ignore_index=True)

    # unload temp variables
    del dfs
    gc.collect()

    # update dtypes (doesn't carry through concat if done on read_csv...?)
    coltypes = {
        "tripduration": "int32",
        "starttime": "datetime64",
        "stoptime": "datetime64",
        "startstationid": "category",
        "startstationname": "category",
        "startstationlatitude": "category",
        "startstationlongitude": "category",
        "endstationid": "category",
        "endstationname": "category",
        "endstationlatitude": "category",
        "endstationlongitude": "category",
        "bikeid": "category",
        "usertype": "category",
        "birthyear": "category",
        "gender": "category",
    }
    clobbered = clobbered.astype(coltypes)
    print(year, "...dtypes converted")

    return clobbered

In [ ]:
def gen_data_files(
    years=[2019], state="NY"
) -> pd.DataFrame:  # what does -> pd.dataframe do?
    """
    Calls clobber_year and writes output to both csv and parquet to `data/`

    :param years: list of years to generate data files for
    :param state: 'NY' or 'NJ'. default 'NY'
    :return: nothing
    """

    gc.collect()
    for year in years:
        # clobber dataframe
        print("clobbering...", year)
        temp_df = clobber_year(year, state)
        print(year, "...clobbered")

        # extract station data [only if uberparquet faisl]
        # perform reverse geocode ---> later
        # create yearly stations dataframe
        # save to file
        # unload related dfs

        # clean dataframe
        # drop NAs? - not implemented atm
        # drop station cols

        # save to files
        exportpath = "data/rides_" + str(year) + ".parquet"
        temp_df.to_parquet(exportpath)
        print(year, "...saved to parquet")

        exportpath = "data/rides_" + str(year) + ".csv"
        temp_df.to_csv(exportpath)
        print(year, "...saved to csv")

        # unload dataframe
        del temp_df
        gc.collect()
        print(year, "...unloaded")